<a href="https://colab.research.google.com/github/TouaziAimenRyad/LargeFileHndlerSpark/blob/master/BIG_DATA_PYSPARK_TOUZI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Setting Up SPARK**

In [ ]:
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
! wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [ ]:
! tar xf /content/spark-3.3.2-bin-hadoop3.tgz

In [ ]:
! pip install -q findspark
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=b19b39ac2ab3545099d8c5c7295bdd84111ea51b8e66d50b4257b277bedef23f
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
# Set up required environment variables

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

import findspark

findspark.init("spark-3.3.2-bin-hadoop3")


In [ ]:
# in order to use avro
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.5 pyspark-shell'


In [ ]:
#spark context object 
from pyspark import SparkContext, SparkConf
configuration = SparkConf().setAppName("Projet").setMaster("local[*]")
sc = SparkContext(conf=configuration)

In [ ]:
#spark session object
from pyspark.sql import SparkSession 
spark=SparkSession.builder.appName('Projet').getOrCreate()

#**Data Prep**

In [ ]:
#data download
!wget https://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz

--2023-03-06 10:07:21--  https://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17329808 (17M) [application/x-gzip]
Saving to: ‘news20.tar.gz’

news20.tar.gz       100%[===================>]  16.53M  3.00MB/s    in 6.0s    

2023-03-06 10:07:27 (2.77 MB/s) - ‘news20.tar.gz’ saved [17329808/17329808]



In [ ]:
#decompress the data 
!tar -xzf news20.tar.gz

In [ ]:
#rdd creation 
data_dir = "20_newsgroup"

atheism_rdd = sc.wholeTextFiles(data_dir + "/alt.atheism/*")
baseball_rdd = sc.wholeTextFiles(data_dir + "/rec.sport.baseball/*")

In [ ]:
#check if the rdd were created succesfully 
print("Number of documents in alt.atheism RDD: ", atheism_rdd.count())
print("first entry of documents in alt.atheism RDD: ", atheism_rdd.first())#doc name in 0 and header and body in 1

print("Number of documents in rec.sport.baseball RDD: ", baseball_rdd.count())
print("first entry of documents in rec.sport.baseball RDD: ", baseball_rdd.first())


Number of documents in alt.atheism RDD:  1000
first entry of documents in alt.atheism RDD:  ('file:/content/20_newsgroup/alt.atheism/49960', 'Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew\nFrom: mathew <mathew@mantis.co.uk>\nNewsgroups: alt.atheism,alt.atheism.moderated,news.answers,alt.answers\nSubject: Alt.Atheism FAQ: Atheist Resources\nSummary: Books, addresses, music -- anything related to atheism\nKeywords: FAQ, atheism, books, music, fiction, addresses, contacts\nMessage-ID: <19930329115719@mantis.co.uk>\nDate: Mon, 29 Mar 1993 11:57:19 GMT\nExpires: Thu, 29 Apr 1993 11:57:19 GMT\nFollowup-To: alt.atheism\nDistribution: world\nOrganization: Mantis Consultants, Cambridge. UK.\nApproved: news-answers-request@m

In [ ]:
#separer le corps de message de l'entete \n\n
def split_header_body(doc):
    header, body = doc[1].split("\n\n", 1) # in 0 we have doc name in 1 we have the content=header+body
    return (doc[0], header, body)

In [ ]:
atheism_rdd = atheism_rdd.map(split_header_body) # doc name in 0 header in 1 body in 2 
baseball_rdd = baseball_rdd.map(split_header_body)
print("Number of documents in alt.atheism RDD: ", atheism_rdd.count())
print("first entry of documents in alt.atheism RDD: ", atheism_rdd.first())

print("Number of documents in rec.sport.baseball RDD: ", baseball_rdd.count())
print("first entry of documents in rec.sport.baseball RDD: ", baseball_rdd.first())


Number of documents in alt.atheism RDD:  1000
first entry of documents in alt.atheism RDD:  ('file:/content/20_newsgroup/alt.atheism/49960', 'Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew\nFrom: mathew <mathew@mantis.co.uk>\nNewsgroups: alt.atheism,alt.atheism.moderated,news.answers,alt.answers\nSubject: Alt.Atheism FAQ: Atheist Resources\nSummary: Books, addresses, music -- anything related to atheism\nKeywords: FAQ, atheism, books, music, fiction, addresses, contacts\nMessage-ID: <19930329115719@mantis.co.uk>\nDate: Mon, 29 Mar 1993 11:57:19 GMT\nExpires: Thu, 29 Apr 1993 11:57:19 GMT\nFollowup-To: alt.atheism\nDistribution: world\nOrganization: Mantis Consultants, Cambridge. UK.\nApproved: news-answers-request@m

In [ ]:
#extraire qlq champs de l'entete
def extract_fields(header):
    fields = {}
    for line in header.split("\n"):
        parts = line.split(": ", 1)
        if len(parts) == 2:
            fields[parts[0]] = parts[1]
    return fields

In [ ]:
# this new rdds contains only the fields they have no body we made a mistake at first and tried using them for 
#the rest of the question but nothing worked so we merged the normal rdds and used 
#the extract function to create sql rows and we letf these as an exp
atheism_fields_rdd = atheism_rdd.map(lambda doc: (doc[0], extract_fields(doc[1])))
baseball_fields_rdd = baseball_rdd.map(lambda doc: (doc[0], extract_fields(doc[1])))

In [ ]:
print("Sample fields from a document in the atheism RDD: ", atheism_fields_rdd.first())
print("Sample fields from a document in the baseball RDD: ", baseball_fields_rdd.first())

Sample fields from a document in the atheism RDD:  ('file:/content/20_newsgroup/alt.atheism/49960', {'Xref': 'cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126', 'Path': 'cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew', 'From': 'mathew <mathew@mantis.co.uk>', 'Newsgroups': 'alt.atheism,alt.atheism.moderated,news.answers,alt.answers', 'Subject': 'Alt.Atheism FAQ: Atheist Resources', 'Summary': 'Books, addresses, music -- anything related to atheism', 'Keywords': 'FAQ, atheism, books, music, fiction, addresses, contacts', 'Message-ID': '<19930329115719@mantis.co.uk>', 'Date': 'Mon, 29 Mar 1993 11:57:19 GMT', 'Expires': 'Thu, 29 Apr 1993 11:57:19 GMT', 'Followup-To': 'alt.atheism', 'Distribution': 'world', 'Organization': 'Mantis Consultants, Cambridge. UK.', 'Approved': 'news-ans

In [ ]:
#6-RDD fusion 
merged_rdd = atheism_rdd.union(baseball_rdd) # fusing rdds that contains doc name header and body in 0 1 2 respectivly 
print("body of the first documents in merged RDD: ", merged_rdd.first()[2])


body of documents in merged RDD:  Archive-name: atheism/resources
Alt-atheism-archive-name: resources
Last-modified: 11 December 1992
Version: 1.0

                              Atheist Resources

                      Addresses of Atheist Organizations

                                     USA

FREEDOM FROM RELIGION FOUNDATION

Darwin fish bumper stickers and assorted other atheist paraphernalia are
available from the Freedom From Religion Foundation in the US.

Write to:  FFRF, P.O. Box 750, Madison, WI 53701.
Telephone: (608) 256-8900

EVOLUTION DESIGNS

Evolution Designs sell the "Darwin fish".  It's a fish symbol, like the ones
Christians stick on their cars, but with feet and the word "Darwin" written
inside.  The deluxe moulded 3D plastic fish is $4.95 postpaid in the US.

Write to:  Evolution Designs, 7119 Laurel Canyon #4, North Hollywood,
           CA 91605.

People in the San Francisco Bay area can get Darwin Fish from Lynn Gold --
try mailing <figmo@netcom.com>.  For net p

In [ ]:
#transformation de rdd pyspark sql row
from pyspark.sql import Row

In [ ]:
def create_row(doc_id, fields, body):
    return Row(doc_id=doc_id, organization=fields.get("Organization", ""), newsgroups=fields.get("Newsgroups", ""),subject=fields.get("Subject",""),Date=fields.get("Date",""),From=fields.get("From",""), lines=fields.get("Lines",""), body=body)

In [ ]:
row_rdd = merged_rdd.map(lambda doc: create_row(doc[0], extract_fields(doc[1]), doc[2])) # creating row from the merged rdd and using the extract functions to extract spsific fields from the header  
print("Number of documents in row RDD: ", row_rdd.count())


Number of documents in row RDD:  2000


In [ ]:
#creating data frame 
df = row_rdd.toDF(),#this creates a tupple this is just for testing and seeing the diffrence between the 
df2 = spark.createDataFrame(row_rdd) # this create a spark dataframe we will continue working with this  functions 
df2.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+
|              doc_id|        organization|          newsgroups|             subject|                Date|                From|lines|                body|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+
|file:/content/20_...|Mantis Consultant...|alt.atheism,alt.a...|Alt.Atheism FAQ: ...|Mon, 29 Mar 1993 ...|mathew <mathew@ma...|  290|Archive-name: ath...|
|file:/content/20_...|Mantis Consultant...|alt.atheism,alt.a...|Alt.Atheism FAQ: ...|Mon, 5 Apr 1993 1...|mathew <mathew@ma...|  646|Archive-name: ath...|
|file:/content/20_...|Technical Univers...|         alt.atheism|   Re: Gospel Dating|Mon, 5 Apr 1993 1...|I3150101@dbstu1.r...|   93|In article <65974...|
|file:/content/20_...|Mantis Consultant...|alt.atheism,alt.p...|Re: un

In [ ]:
!pip install fastavro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.3 MB/s eta 0:00:00


In [ ]:
!pip install pandavro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.3 MB/s eta 0:00:00
  Created wheel for pandavro: filename=pandavro-1.7.1-py3-none-any.whl size=5688 sha256=da6726d13eb8c7a39a4aff79560bfa78c597805179c4267e031a48dfd7294cdc
  Stored in directory: /root/.cache/pip/wheels/85/cc/15/480a3cfa1a9fc49e2af9bde7437bd6c8c0265f17e61f32672b
Successfully built pandavro
  Attempting uninstall: fastavro
    Found existing installation: fastavro 1.7.2
    Uninstalling fastavro-1.7.2:
      Successfully uninstalled fastavro-1.7.2


In [ ]:
#save format avro 
from fastavro import writer, reader, parse_schema
import numpy as np
import pandas as pd
import pandavro as pdx


In [ ]:
avro_schema ={
   'doc': '20 news group',
   'namespace': 'test',
   'type': 'record',
   'name': 'news',
   'fields': [
      {'name': 'doc_id', 'type': 'string'},
      {'name': 'organization', 'type': 'string'},
      {'name': 'newsgroups', 'type': 'string'},
      {'name': 'body', 'type': 'string'}
   ]
}

avro_schema=parse_schema(avro_schema)
type(avro_schema)

dict

In [ ]:
#1-method: converting to pandas then records before writing in avro file !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df_pandas=df2.toPandas() 
records = df_pandas.to_dict('records')
#df_pandas.head()

In [ ]:
with open('sample_df.avro', 'wb') as out:
    writer(out,avro_schema,records)


In [ ]:
#2-method:writng the pandas data frame directly into avro file using pandasavro
pdx.to_avro("sample_df2.avro", df_pandas)

In [ ]:
#save parquet format
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

table = pa.Table.from_pandas(df_pandas)
pq.write_table(table, 'sample_df.parquet')

#**Analyse descriptive**

In [ ]:
#Vérifier qu’on a bien deux catégories différentes de documents
df2.createOrReplaceTempView("documents")

spark.sql("SELECT documents.newsgroups, COUNT(*) as count FROM documents GROUP BY documents.newsgroups").show()



+--------------------+-----+
|          newsgroups|count|
+--------------------+-----+
|talk.religion.mis...|    4|
|talk.religion.mis...|    2|
|alt.atheism,talk....|   20|
|alt.atheism,soc.m...|    8|
|talk.religion.mis...|    4|
|alt.atheism,soc.c...|    2|
|alt.atheism,alt.f...|    1|
|alt.atheism,alt.p...|    2|
|alt.atheism,talk....|    5|
|alt.atheism,rec.s...|    3|
|talk.abortion,alt...|   94|
|alt.atheism,talk....|    2|
|alt.atheism,talk....|    1|
|alt.atheism,alt.a...|    7|
|alt.atheism,talk....|   92|
|         alt.atheism|  734|
|soc.culture.arabi...|    2|
|soc.culture.arabi...|    1|
|rec.scouting,soc....|    1|
|alt.slack,talk.re...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
#Donner le nombre d'organisations différentes
spark.sql("SELECT COUNT(DISTINCT organization) as count FROM documents").show()

+-----+
|count|
+-----+
|  486|
+-----+



In [ ]:
#Suivant les champs extraits à la partie 3, donner d’autres statistiques descriptives (nombre, moyenne, etc)
spark.sql("SELECT AVG(lines) as avg_lines_body FROM documents").show()

+-----------------+
|   avg_lines_body|
+-----------------+
|36.48294884653962|
+-----------------+



In [ ]:
# authors and how many docs they wrote 
df2.groupBy("FROM").count().show()

+--------------------+-----+
|                FROM|count|
+--------------------+-----+
|bobbe@vice.ICO.TE...|   24|
|jcopelan@nyx.cs.d...|    5|
|jgacker@news.gsfc...|    1|
|pharvey@quack.kfu...|    3|
|davidk@welch.jhu....|    2|
|ingles@engin.umic...|    6|
|p00261@psilink.co...|    1|
|a137490@lehtori.c...|    5|
|timmbake@mcl.ucsb...|    6|
|sandvik@newton.ap...|   25|
|cfaehl@vesta.unm....|    3|
|rm03@ic.ac.uk (Mr...|    1|
|simon@dcs.warwick...|    3|
|agrino@enkidu.mic...|    1|
|strom@Watson.Ibm....|    1|
|bevans@carina.unm...|    1|
|nyeda@cnsvax.uwec...|    8|
|wolfe@wolves.Durh...|    1|
|jbrown@batman.bmd...|    8|
|nancyo@shnext15.u...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
#nbr of doc in each organization
df2.groupBy("organization").count().show()

+--------------------+-----+
|        organization|count|
+--------------------+-----+
|Brownian Motion Inc.|    1|
|           Summit NJ|    1|
|Software Engineer...|    4|
|Sea Change Corpor...|    1|
|University of Was...|    8|
|California Instit...|   65|
|Kupajava, East of...|    9|
|Express Access On...|    3|
|Harlequin Ltd, Ca...|    1|
|California State ...|    1|
|Salvation Army Dr...|    3|
|California Instit...|    3|
|Johns Hopkins Uni...|    7|
|Mantis Consultant...|   40|
|    Imperial College|    1|
|Louisiana Tech Un...|    2|
|Nyx, Public Acces...|    6|
|Penn State Univer...|   13|
|Society for Putti...|    7|
|Computing Service...|    3|
+--------------------+-----+
only showing top 20 rows



In [ ]:
#stats on nb lines 
df2.describe("Lines").show()

+-------+------------------+
|summary|             Lines|
+-------+------------------+
|  count|              2000|
|   mean| 36.48294884653962|
| stddev|48.791372734500925|
|    min|                  |
|    max|                99|
+-------+------------------+



In [ ]:
#spark.sql("DROP VIEW religion")

In [ ]:
# creation of vue that only contains religion docs
spark.sql("CREATE TEMP VIEW religion AS SELECT newsgroups FROM documents WHERE newsgroups LIKE '%alt.atheism%'")

DataFrame[]

In [ ]:
# nombre of document for athesim
spark.sql("SELECT count(newsgroups) as nb_alt_atheism FROM religion").show()

+--------------+
|nb_alt_atheism|
+--------------+
|          1000|
+--------------+



#**Transformation du texte**

In [ ]:
# transform in Row sql the entire document not just the body and few fields . this wiil be applied on the merged rdd that contains the entire doc 0 name 1 header 2 body
def allToRow1(doc):
  return Row(document=doc[0]+doc[1]+doc[2]) # i might remove doc id from the row cause i think it might affect the clustering 

In [ ]:
def allToRow(doc):
  return Row(document=doc[1]+doc[2]) # we removed doc id and we continue with this function

In [ ]:
new_row_rdd=merged_rdd.map(allToRow)
print("printing the first element to test in it's correct : ",new_row_rdd.first())

printing the first element to test in it's correct :  Row(document='Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew\nFrom: mathew <mathew@mantis.co.uk>\nNewsgroups: alt.atheism,alt.atheism.moderated,news.answers,alt.answers\nSubject: Alt.Atheism FAQ: Atheist Resources\nSummary: Books, addresses, music -- anything related to atheism\nKeywords: FAQ, atheism, books, music, fiction, addresses, contacts\nMessage-ID: <19930329115719@mantis.co.uk>\nDate: Mon, 29 Mar 1993 11:57:19 GMT\nExpires: Thu, 29 Apr 1993 11:57:19 GMT\nFollowup-To: alt.atheism\nDistribution: world\nOrganization: Mantis Consultants, Cambridge. UK.\nApproved: news-answers-request@mit.edu\nSupersedes: <19930301143317@mantis.co.uk>\nLines: 290Archive-name:

In [ ]:
#creating data frame for the entire document for all entiries 
all_doc_df=spark.createDataFrame(new_row_rdd)
all_doc_df.show()

+--------------------+
|            document|
+--------------------+
|Xref: cantaloupe....|
|Xref: cantaloupe....|
|Newsgroups: alt.a...|
|Xref: cantaloupe....|
|Xref: cantaloupe....|
|Newsgroups: alt.a...|
|Path: cantaloupe....|
|Newsgroups: alt.a...|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Xref: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
+--------------------+
only showing top 20 rows



In [ ]:
# creating the word list using tokenizer
from pyspark.ml.feature import Tokenizer

In [ ]:
tokenizer = Tokenizer(inputCol="document", outputCol="word_list")
words_df = tokenizer.transform(all_doc_df)
words_df.show()

+--------------------+--------------------+
|            document|           word_list|
+--------------------+--------------------+
|Xref: cantaloupe....|[xref:, cantaloup...|
|Xref: cantaloupe....|[xref:, cantaloup...|
|Newsgroups: alt.a...|[newsgroups:, alt...|
|Xref: cantaloupe....|[xref:, cantaloup...|
|Xref: cantaloupe....|[xref:, cantaloup...|
|Newsgroups: alt.a...|[newsgroups:, alt...|
|Path: cantaloupe....|[path:, cantaloup...|
|Newsgroups: alt.a...|[newsgroups:, alt...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Xref: cantaloupe....|[xref:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, cantaloup...|
|Path: cantaloupe....|[path:, ca

In [ ]:
#Creating a vectoriel representation using HashingTf
from pyspark.ml.feature import HashingTF

In [ ]:
hashingTF = HashingTF(inputCol="word_list", outputCol="features")
tf_df = hashingTF.transform(words_df)

#**Grouper les documents qui ont des représentations vectorielles proches**

In [ ]:
#k means
from pyspark.ml.clustering import KMeans

# Create a KMeans instance with k=2 clusters
kmeans = KMeans().setK(2).setSeed(1)

# Train the model on the features column
model = kmeans.fit(tf_df.select("features"))

# Get the cluster assignments for each document
predictions = model.transform(tf_df)
predictions.show()

+--------------------+--------------------+--------------------+----------+
|            document|           word_list|            features|prediction|
+--------------------+--------------------+--------------------+----------+
|Xref: cantaloupe....|[xref:, cantaloup...|(262144,[154,521,...|         0|
|Xref: cantaloupe....|[xref:, cantaloup...|(262144,[409,489,...|         0|
|Newsgroups: alt.a...|[newsgroups:, alt...|(262144,[19,321,8...|         0|
|Xref: cantaloupe....|[xref:, cantaloup...|(262144,[6193,853...|         0|
|Xref: cantaloupe....|[xref:, cantaloup...|(262144,[4971,619...|         0|
|Newsgroups: alt.a...|[newsgroups:, alt...|(262144,[453,2306...|         0|
|Path: cantaloupe....|[path:, cantaloup...|(262144,[1818,619...|         0|
|Newsgroups: alt.a...|[newsgroups:, alt...|(262144,[3530,421...|         0|
|Path: cantaloupe....|[path:, cantaloup...|(262144,[2564,293...|         0|
|Path: cantaloupe....|[path:, cantaloup...|(262144,[6193,636...|         0|
|Path: canta

In [ ]:
#seeing the clsuters 
predictions.createOrReplaceTempView("test")
spark.sql("SELECT count(*) as nb_doc_in_each_cluster,prediction as type_of_doc from test group by prediction").show()

+----------------------+-----------+
|nb_doc_in_each_cluster|type_of_doc|
+----------------------+-----------+
|                     5|          1|
|                  1995|          0|
+----------------------+-----------+



#**Partie optionel**

In [ ]:
#1. Pondérer les mots avec la formule Tf-Idf (avant KMeans)

from pyspark.sql.functions import split
df3 = words_df.withColumn("word_list", split(words_df["document"], " ")) 
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="word_list", outputCol="rawFeatures", vocabSize=10000)
cvModel = cv.fit(df3)
tf = cvModel.transform(df3).select("document", "rawFeatures")
from pyspark.ml.feature import IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(tf)
tfidf = idfModel.transform(tf).select("document", "features")

In [ ]:
#2. Normaliser les vecteurs représentant les documents (avant KMeans).

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["features"], outputCol="vectors")
df_vectors = assembler.transform(tfidf).select("document", "vectors")
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="vectors", outputCol="features", p=2.0)
df_norm = normalizer.transform(df_vectors).select("document", "features")
df_norm.show()

+--------------------+--------------------+
|            document|            features|
+--------------------+--------------------+
|Xref: cantaloupe....|(10000,[0,1,2,3,4...|
|Xref: cantaloupe....|(10000,[0,1,2,3,4...|
|Newsgroups: alt.a...|(10000,[0,1,2,3,4...|
|Xref: cantaloupe....|(10000,[0,1,2,3,4...|
|Xref: cantaloupe....|(10000,[0,1,2,3,5...|
|Newsgroups: alt.a...|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Newsgroups: alt.a...|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[1,3,6,10,...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,3,4,5...|
|Xref: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|
|Path: cantaloupe....|(10000,[0,

In [ ]:
from pyspark.ml.clustering import KMeans

# Create a KMeans instance with k=2 clusters
opt_kmeans = KMeans().setK(2).setSeed(1)

# Train the model on the features column
opt_model = opt_kmeans.fit(df_norm.select("features"))

# Get the cluster assignments for each document
opt_predictions = opt_model.transform(df_norm)
opt_predictions.show()

+--------------------+--------------------+----------+
|            document|            features|prediction|
+--------------------+--------------------+----------+
|Xref: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Xref: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Newsgroups: alt.a...|(10000,[0,1,2,3,4...|         0|
|Xref: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Xref: cantaloupe....|(10000,[0,1,2,3,5...|         0|
|Newsgroups: alt.a...|(10000,[0,1,2,3,4...|         0|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Newsgroups: alt.a...|(10000,[0,1,2,3,4...|         0|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Path: cantaloupe....|(10000,[1,3,6,10,...|         0|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Path: cantaloupe....|(10000,[0,1,3,4,5...|         0|
|Xref: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Path: cantaloupe....|(10000,[0,1,2,3,4...|         0|
|Path: can

In [ ]:
#seeing the clsuters 
opt_predictions.createOrReplaceTempView("opt_test")
spark.sql("SELECT count(*) as nb_doc_in_each_cluster,prediction as type_of_doc from opt_test group by prediction").show()

+----------------------+-----------+
|nb_doc_in_each_cluster|type_of_doc|
+----------------------+-----------+
|                    85|          1|
|                  1915|          0|
+----------------------+-----------+

